# 使用TensorFlow解决iris问题

Eager Execution可以使TensorFlow立即评估各项操作，并返回具体的值，不会创建稍后执行的计算图。

In [1]:
import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()

print("Tensorflow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

/Users/lazycat/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/lazycat/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Tensorflow version: 1.10.1
Eager execution: True


下载数据集，并储存

In [2]:
train_data_url = "http://download.tensorflow.org/data/iris_training.csv"
train_data_fp = tf.keras.utils.get_file(fname=os.path.basename(train_data_url), origin=train_data_url)

print("Local copy of the dataset file: {}".format(train_data_fp))

Local copy of the dataset file: /Users/lazycat/.keras/datasets/iris_training.csv


In [3]:
def parse_csv(line):
    defaults = [[0.], [0.], [0.], [0.], [0]]
    parsed_line = tf.decode_csv(line, defaults)
    # 提取前4个特征值
    features = tf.reshape(parsed_line[:-1], shape=(4,))
    # 提取最后一个标签值
    label = tf.reshape(parsed_line[-1], shape=())
    return features, label

In [5]:
train_data = tf.data.TextLineDataset(train_data_fp)

# 跳过第一行Hdeader
train_data = train_data.skip(1)

# 对每一行进行处理
train_data = train_data.map(parse_csv)

# 随机化数据
train_data = train_data.shuffle(buffer_size=1000)

train_data = train_data.batch(32)

features, label = iter(train_data).next()
print("example features:", features[0])
print("example label:", label[0])

example features: tf.Tensor([5.  2.3 3.3 1. ], shape=(4,), dtype=float32)
example label: tf.Tensor(1, shape=(), dtype=int32)


## 使用神经网络来解决分类问题

使用密集型神经网络，隐藏层是密集层。

In [7]:
# 定义模型，注意第一个密集层需要指定输入规模
# 这里有4个特征值，所以输入规模是4
# 输出密集层的参数是3，因为有3种花的种类
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(3)
])

## 训练模型

在训练模型之前，需要计算模型的损失。这样可以衡量模型的预测结果和预期标签有多大偏差。通过这个偏差可以调整模型。

同时还要定义梯度，梯度用于在训练的时候优化模型

In [8]:
def loss(model, x, y):
    y_ = model(x)
    return tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)


def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_val = loss(model, inputs, targets)
    return tape.gradient(loss_val, model.variables)

In [ ]:
# 随机梯度